In [64]:
import graph_characteristics_lib as gc

In [65]:
names = [ ["dnc","dnc","crimson","d"] , ["fb-forum", "fb", "green","d"], ["talk_eo", "eo" ,"olive","d"],
          ["email-eu3", "eu3", "yellow","d"], ["email-eu2", "eu2", "magenta","d"], 
          ["racoon", "rac", "brown","u"], ["primate","pri",  "blue","u"],  ["workplace_2013", "wp", "black","u"],
           ["ht09", "ht","red","u"], ["weaver", "wea",  "pink","u"] ]

In [66]:
def find_sep(x):
    for s in x:
        if s not in ['0','1','2','3','4','5','6','7','8','9']:
            return s
    return None

def read_graph(path,s, dire):
    g = set()
    f = open(path+s+".csv", "r")
    sep = find_sep(f.readline())
    f.close()
    f = open(path+s+".csv", "r")
    for x in f:
        x = x[:-1]
        r = x.split(sep)
        u,v,t = r[0],r[1],r[2]
        if v != u:
            t = float(t)
            g.add( (u,v,int(t)) )
            if dire == "u":
                g.add( (v,u,int(t)) )
    return g

def write_graph(path, s, g):
    f = open(path+s+".csv", "w")
    for x in g:
        f.write( str(x) )
    f.close()

In [67]:
fold_d = "datasets/networks/"
for i in range(len(names)):
    cur = fold_d + names[i][0]+".csv"
    g = read_graph(fold_d,names[i][0], names[i][-1])
#     print(names[i][0], nb_nodes(cur),nb_time_events(cur),nb_edges(cur))
    count = (names[i][-1],len(gc.nodes(g)))
#     count = nb_time_events(cur)
    names[i].insert(0,count)
print(names)
names.sort(reverse = True)
names = [ names[i][1:] for i in range(len(names))  ]
print(names)
for i in range(len(names)):
    g = read_graph(fold_d,names[i][0], names[i][-1])
    print(names[i][0], "& ",names[i][1], "&" ,names[i][-1],   "& $", len(gc.nodes(g)), "$ & $", len(gc.events(g)), "$ & $" ,len(g), "$ \\\\")

[[('d', 1866), 'dnc', 'dnc', 'crimson', 'd'], [('d', 899), 'fb-forum', 'fb', 'green', 'd'], [('d', 7288), 'talk_eo', 'eo', 'olive', 'd'], [('d', 89), 'email-eu3', 'eu3', 'yellow', 'd'], [('d', 162), 'email-eu2', 'eu2', 'magenta', 'd'], [('u', 24), 'racoon', 'rac', 'brown', 'u'], [('u', 25), 'primate', 'pri', 'blue', 'u'], [('u', 92), 'workplace_2013', 'wp', 'black', 'u'], [('u', 113), 'ht09', 'ht', 'red', 'u'], [('u', 445), 'weaver', 'wea', 'pink', 'u']]
[['weaver', 'wea', 'pink', 'u'], ['ht09', 'ht', 'red', 'u'], ['workplace_2013', 'wp', 'black', 'u'], ['primate', 'pri', 'blue', 'u'], ['racoon', 'rac', 'brown', 'u'], ['talk_eo', 'eo', 'olive', 'd'], ['dnc', 'dnc', 'crimson', 'd'], ['fb-forum', 'fb', 'green', 'd'], ['email-eu2', 'eu2', 'magenta', 'd'], ['email-eu3', 'eu3', 'yellow', 'd']]
weaver &  wea & u & $ 445 $ & $ 23 $ & $ 2666 $ \\
ht09 &  ht & u & $ 113 $ & $ 5246 $ & $ 41636 $ \\
workplace_2013 &  wp & u & $ 92 $ & $ 7104 $ & $ 19654 $ \\
primate &  pri & u & $ 25 $ & $ 19 $ &

In [126]:
def find_names(x):
    s = ""
    for i in range(len(x)):
        e = x[i]
        if (ord(e) >= ord('a') and ord('z') >= ord(e)) or (ord(e) >= ord('A') and ord('Z') >= ord(e)) or (ord(e)>= ord('0') and ord('9')>=ord(e)) or e == '-' or e == '_':
            s += e
        else:
            break
    return s,i
        
def read_next_number(x,i):
    num = ""
    while (i < len(x)) and (not ((ord(x[i]) >= ord('0') and ord('9') >= ord(x[i]) ))):
        i += 1
    if i >= len(x):
        return False
    
    #first part
    while (ord(x[i]) >= ord('0') and ord('9') >= ord(x[i]) ):
        num += x[i]
        i += 1
    if (x[i] != '.'):
        return float(num), i+1
    else:
        num += '.'
        i += 1
        
    #second part
    while (ord(x[i]) >= ord('0') and ord('9') >= ord(x[i]) ):
        num += x[i]
        i += 1
    return float(num), i+1
    
def add_line(x,st = 0):
    l = []
    nb,ind = read_next_number(x,st)
    l.append(nb)
    while True:
        res = read_next_number(x,ind)
        if type(res) != bool:
            nb,ind = res
            l.append(nb)
        else:
            break
    return l
    
def read_tex(s):
    f = open(s,"r")
    d = dict()
    i = 0
    for x in f:
        av = []
        dev = []
        x = x[:-1]
        if i%2 == 0:
            nam,ind = find_names(x)
            d[nam] = dict()
        if i%2 == 0:
            d[nam]["av"] = add_line(x,ind)  
        else:
            d[nam]["dev"] = add_line(x)
        i += 1
    f.close()
    return d

In [131]:
def renorm_d(names,d):
    for i in range(len(names)):
        cur = fold_d + names[i][0]+".csv"
        g = read_graph(fold_d,names[i][0], names[i][-1])
        for j in range(len(d[names[i][1]]["av"])):
            d[names[i][1]]["av"][j] = d[names[i][1]]["av"][j]/len(g)
        for j in range(len(d[names[i][1]]["dev"])):
            d[names[i][1]]["dev"][j] = d[names[i][1]]["dev"][j]/len(g)
    return d

In [161]:
d = read_tex("table_charac__clus.tex")

In [167]:
d = read_tex("table_charac__tri.tex")

In [168]:
d = renorm_d(names,d)

In [169]:
def format_d(d, names):
    for e in names:
        print(e[1]+" & ", end = "")
        val = d[e[1]]["av"][0]
        closest = min( list(map( lambda x : abs(val-x) ,d[e[1]]["av"][1:])) )
        for j in range(len((d[e[1]]["av"]))):
            ee = d[e[1]]["av"][j]
            dev = d[e[1]]["dev"][j]
            if abs(val-ee) == closest and j > 0:
                print( '$\mathbf{ '+'{:.2f}'.format(ee)+' ('+ '{:.2f}'.format(dev) +') } $ & ', end = "" )
            else:
                if j>0:
                    print( '$'+'{:.2f}'.format(ee)+' ( '+ '{:.2f}'.format(dev) +')$ & ', end = "" )
                else:
                    print( '$'+'{:.2f}'.format(ee)+'$ & ', end = "" )
                    
        print("\\\\")

In [170]:
format_d(d, names)

wea & $1.55$ & $\mathbf{ 1.54 (0.00) } $ & $1.43 ( 0.01)$ & $0.96 ( 0.02)$ & $0.06 ( 0.01)$ & \\
ht & $0.45$ & $\mathbf{ 0.45 (0.05) } $ & $0.45 ( 0.27)$ & $2.21 ( 2.03)$ & $2.13 ( 1.42)$ & \\
wp & $2.38$ & $\mathbf{ 2.38 (0.00) } $ & $\mathbf{ 2.38 (0.00) } $ & $1.43 ( 2.65)$ & $0.96 ( 1.04)$ & \\
pri & $27.73$ & $\mathbf{ 27.73 (0.06) } $ & $27.74 ( 0.03)$ & $27.35 ( 1.43)$ & $26.36 ( 1.30)$ & \\
rac & $7.20$ & $\mathbf{ 7.20 (0.00) } $ & $7.19 ( 0.28)$ & $7.13 ( 2.40)$ & $6.97 ( 2.02)$ & \\
eo & $0.99$ & $\mathbf{ 0.97 (1.12) } $ & $0.96 ( 1.39)$ & $0.00 ( 0.00)$ & $0.00 ( 0.00)$ & \\
dnc & $2.14$ & $\mathbf{ 2.12 (0.93) } $ & $2.06 ( 0.82)$ & $0.05 ( 0.01)$ & $0.05 ( 0.00)$ & \\
fb & $1.32$ & $\mathbf{ 1.28 (0.02) } $ & $1.05 ( 0.05)$ & $0.50 ( 0.03)$ & $0.51 ( 0.02)$ & \\
eu2 & $0.33$ & $\mathbf{ 0.33 (0.48) } $ & $0.31 ( 0.54)$ & $1.54 ( 0.47)$ & $1.59 ( 0.55)$ & \\
eu3 & $5.95$ & $\mathbf{ 5.89 (0.70) } $ & $5.19 ( 4.35)$ & $6.67 ( 1.69)$ & $6.90 ( 1.82)$ & \\
